In [1]:
# Importing Useful Libraries

import numpy as np
import matplotlib.pyplot as plt
import random as rd
import time
import copy

In [7]:
# Producting Connection Matrix for Single Layer

def Amat(size,num):
    matrix = np.zeros((size,size))
    count=0
    while count!=num:
        x,y=np.random.randint(size),np.random.randint(size)
        if matrix[x][y]==0 and x != y:
            matrix[x][y] = matrix[y][x]=1
            count = count+1
    return matrix

# Producing Connection Matrix for arbitrary number of layers

def LAmat(L, Size, Connec):
    PreAL = []
    for i in range(L):
        PreAL.append(Amat(Size, Connec))
    return np.array(PreAL)

In [3]:
# Function to assign randomly straties to individual of population denoted by indices

def Strat(Size, Frac):
    Pop = np.zeros(Size)
    for i in range(int(Frac*Size)):
        Pop[i] = 1
    rd.shuffle(Pop)
    return Pop

# Function to assign strategy to individuals in different layers

def LStrat(L, Size, Frac):
    PreP = []
    for i in range(L):
        PreP.append(Strat(Size, Frac))
    return np.array(PreP)

In [4]:
# Function to collect payoff for each indivial in the population after interaction in single layer

def Payoff(population, Amatrix, Mmatrix, n):
    PO = np.zeros(len(population))
    
    for i in range(len(population)):
        p = 0
        for j in range(len(population)):
            if   Amatrix[n][i][j] == 1 and population[i] == 0 and population[j] == 0:
                p += Mmatrix[0]
                
            elif Amatrix[n][i][j] == 1 and population[i] == 0 and population[j] == 1:
                p += Mmatrix[1]
                
            elif Amatrix[n][i][j] == 1 and population[i] == 1 and population[j] == 0:
                p += Mmatrix[2]
                
            elif Amatrix[n][i][j] == 1 and population[i] == 1 and population[j] == 1:
                p += Mmatrix[3]
        PO[i] = p
                
    return np.array(PO)

# Function to collect payoff for each indivial in the population after interaction in all layers

def LPayoff(LPopu, Amatrix, Mmatrix):
    PrePO = np.zeros(len(LPopu[0]))
    for n in range(len(LPopu)):
        PO = Payoff(LPopu[n], Amatrix, Mmatrix, n)
        PrePO = PrePO + PO
    return PrePO

In [5]:
# Function to obtain probability of changing strategy

def PIE(i, j, S, T, Amatrix, Pmatrix):
    Ki = 0
    Kj = 0
    for o in range(len(Amatrix)):
        kil = np.sum(Amatrix[o][i])
        kjl = np.sum(Amatrix[o][j])
        Ki = Ki + kil
        Kj = Kj + kjl
    Numer = Pmatrix[j] - Pmatrix[i]
    Denom = max(Ki, Kj)*( max(1, T) - min(0, S) )
    return Numer/Denom

In [6]:
# Function to do one round of simulation for a given population, network and payoff matrix

def Simulation(L, Iter, population, Amatrix, Mmatrix, DP):
    Copy_Pop = copy.deepcopy(population)
    FreqL = np.zeros([L, Iter + 1]) # Array to collect frequency of defector after each generation in different layers
    
    for i in range(len(Copy_Pop)):
        fq = np.sum(Copy_Pop[i])/len(Copy_Pop[i])
        FreqL[i][0] = fq
        
    
    for n in range(Iter):
        PO1 = LPayoff(Copy_Pop, Amatrix, Mmatrix) # Collecting Payoff

        for i in range(len(Copy_Pop[0])):
            lr = rd.choice([l for l in range(len(Copy_Pop))])
            
            Neighbr = []
            for z in range(len(Amatrix[lr][i])):
                if Amatrix[lr][i][z] == 1:
                    Neighbr.append(z)
            try:
                jr = rd.choice(Neighbr)
                if PO1[i] < PO1[jr] and Copy_Pop[lr][i] != Copy_Pop[lr][jr]  and rd.random() < PIE(i, jr, Mmatrix[1], Mmatrix[2], Amatrix, PO1):
                    Copy_Pop[lr][i] = Copy_Pop[lr][jr]
            except:
                None
                
        print(DP,":" ,"%3.4f" %((n/Iter)*100), '%', end="\r") # To display progress of simulation
        time.sleep(10**(-3))
        
        for i in range(len(Copy_Pop)):
            fq = np.sum(Copy_Pop[i])/len(Copy_Pop[i])
            FreqL[i][n+1] = fq # Appending frequencies to np array

    return Copy_Pop, FreqL

In [8]:
# Function to create values of pair of value S and T

def ST_value(N1):
    T1 = np.linspace(0, 2, N1)
    S1 = np.linspace(-1, 1, N1)
    Y1 = []
    for i in range(N1):
        Z1 = []
        for j in range(N1):
            X1 = [S1[i], T1[j]]
            Z1.append(X1)
        Y1.append(Z1)
    return np.array(Y1)

In [9]:
# Function to run the simulation for different different payoff matrices

def Sim_Loop(L, NN, Popu, Amatrix, T0, TG):
    
    YY = ST_value(NN)
    
    for s in range(NN):
        for t in range(NN):
            Pick = YY[s][t]
            MM = np.array([1, Pick[0], Pick[1], 0]) # Creating new payoff matrix
            PP, FF = Simulation(L, T0 + TG, Popu, Amatrix, MM, (s, t)) # Doing Simulation
            np.savetxt('Data_S'+str(s)+'_T'+str(t)+'.csv' , FF, delimiter=',') # Exporting defector frequencies to a csv file for further analysis 
            
    return print('Done Duna Done Done')

# ================================

In [10]:
# Running the Simulation
Sim_Loop(5, 21, S0, A0, 10000, 2000)